In [ ]:
import pandas as pd

# 1. Carga de datos

In [ ]:
df_bd = pd.read_excel('bd.xlsx')
df_resultados = pd.read_excel('resultados_completos_v4.xlsx')

# 2. Aplicar filtrar a datos

In [ ]:
def generar_combinaciones(fila):
    nombres = [nombre for nombre in fila if isinstance(nombre, str)]
    combinaciones = []
    
    # Combinaciones de nombres completos
    # NOMBRE: JUAN CARLOS GARCIA PEREZ
    combinaciones.append(" ".join(nombres)) # JUAN CARLOS GARCIA PEREZ
    
    # Combinaciones de iniciales y nombres completos
    # NOMBRE: JUAN CARLOS GARCIA PEREZ
    for i in range(1, len(nombres)):
        combinaciones.append(" ".join([nombres[0]] + [nombre[0] + "." for nombre in nombres[1:i+1]] + nombres[i+1:])) # JUAN C. GARCIA
        combinaciones.append(" ".join([nombres[0][0] + "."] + [nombre[0] + "." for nombre in nombres[1:i+1]] + nombres[i+1:])) # J. C. GARCIA
    
    # Combinaciones de nombres y apellidos
    # NOMBRE: JUAN CARLOS GARCIA PEREZ
    if len(nombres) > 2:
        combinaciones.append(f"{nombres[0]} {nombres[2]}") # JUAN GARCIA
        if len(nombres) > 3:
            combinaciones.append(f"{nombres[0]} {nombres[2]} {nombres[3]}") # JUAN GARCIA PEREZ
        combinaciones.append(f"{nombres[0][0]}. {nombres[2]}") # J. GARCIA
        if len(nombres) > 3:
            combinaciones.append(f"{nombres[0]} {nombres[2]} {nombres[3][0]}") # JUAN GARCIA P
            combinaciones.append(f"{nombres[0]} {nombres[2]} {nombres[3][0]}.") # JUAN GARCIA P
            combinaciones.append(f"{nombres[0][0]}. {nombres[2]} {nombres[3]}") # J. GARCIA PEREZ
    
    # Combinaciones con apellidos compuestos
    if len(nombres) > 3:
        # NOMBRE: JUAN CARLOS GARCIA PEREZ
        apellido_compuesto1 = f"{nombres[2]}-{nombres[3]}" # GARCIA-PEREZ
        combinaciones.append(f"{nombres[0]} {apellido_compuesto1}") # JUAN CARLOS GARCIA-PEREZ
        combinaciones.append(f"{nombres[0][0]}. {apellido_compuesto1}") # J. GARCIA-PEREZ
        combinaciones.append(f"{nombres[0]} {nombres[1][0]}. {apellido_compuesto1}") # JUAN C. GARCIA-PEREZ
        combinaciones.append(f"{nombres[0][0]}. {nombres[1][0]}. {apellido_compuesto1}") # J. C. GARCIA-PEREZ
    
    return combinaciones

In [ ]:
# Separar los nombres de la base de datos
df_nombres_separados = df_resultados["Nombre BD"].str.split(" ", expand=True)
# Aplicar la función a cada fila del DataFrame
todas_combinaciones = df_nombres_separados.apply(generar_combinaciones, axis=1)

In [ ]:
# Crear el diccionario
diccionario_nombres = {}

# Aplicar la función a cada fila del DataFrame
for index, fila in df_nombres_separados.iterrows():
    combinaciones = generar_combinaciones(fila) # Generar combinaciones
    clave = f"{fila[0]} {fila[2]}" if len(fila) > 2 else fila[0] # Clave del diccionario
    diccionario_nombres[clave] = combinaciones # Agregar combinaciones al diccionario
# OUPUT: {NOMBRE APELLIDO: [COMBINACIONES]} (Ejemplo: {"JUAN GARCIA": ["JUAN GARCIA", "J. GARCIA"]})

In [ ]:
def verificar_coincidencia_combinacion_nombres(row):
    search_name = row['Nombre BUSQUEDA']  # Nombre de búsqueda
    name = row['Nombre SCOPUS']  # Nombre original
    
    # Obtener combinaciones del diccionario para el nombre de búsqueda
    combinaciones = diccionario_nombres.get(search_name, [])
    
    # Verificar si el 'Name' está en las combinaciones generadas
    if name in combinaciones:
        return name
    return None

# 3. Exportar datos

In [ ]:
# Aplicar la función a cada registro del DataFrame
df_resultados['Punto Combinacion Nombres'] = df_resultados.apply(verificar_coincidencia_combinacion_nombres, axis=1)
# Exportar resultados
df_resultados.to_excel('resultados_completos_v5.xlsx', index=False)